- Decide on the number of episodes pr. podcast to retrieve.

- Be able to login to the spotify dashboard I and use our own credentials to access the API.

- If a podcaster has questionable descriptions to their podcast, what then? It will appear with almost no "outgoing" links in the network.

- Create dictionary of "podcasters": "podcast name" to be able to accurately find the correct podcast linked to a guests' name, if that name is indeed associated with a podcaster (from a list of podcasters :: Wikipedia). We assume that this list is enough and up to date. For example Brian Redban who is the co-host of "Kill Tony" does not appear on the list. But when the main host will be mentioned somewhere, the podcast will be included.

- Be able to determine all the guests appearing in the episodes of the shows from the descriptions (if we do not go forward with transcripts in any matter). The guests will not have a node in the network, but will be used to make temporal analyses off of the release dates of the podcast episodes, and the guests appearing, to try and find podcasters who are "frontrunners". Thus a given guest should be given a score depending on the number of times they have been on podcasts, as a value of how popular they are. If a podcaster then is amongst the firsts to host this particular guest, they too should be granted a high score.

- Use API to generate the network and extract all the relevant information (guest names, podcasters, when did guests appear).

- How long should the network keep building for. At some point the list might be emptied on its own, but perhaps we should prevent it from running this far. Building on this, we need to make sure that the network does not build towards itself. We start in the JRE podcast, where we find other podcasters that have appeared here. If Joe Rogan then appears on these podcasters podcasts, then it should not loop through the JRE again.

In [ ]:
## Fun and games with Anders

import requests
import base64

import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

def extract_show_id_from_url(url):
    if 'open.spotify.com/show/' in url:
        parts = url.split('/')
        
        show_id = parts[-1]
        return show_id
    else:
        return None


client_id='aa0207fb75aa45eaba50df2d8402a464'
client_secret='42e3594f4e86467a89e27bc2f1d32977'

show_id = '4rOoJ6Egrf8K2IrywzwOMk'
url = f'https://api.spotify.com/v1/shows/{show_id}/episodes?offset=1&limit=20'

auth_header = base64.b64encode(f'{client_id}:{client_secret}'.encode()).decode('utf-8')

headers = {
    'Authorization': f'Basic {auth_header}'
}

payload = {
    'grant_type': 'client_credentials'
}
response = requests.post('https://accounts.spotify.com/api/token', data=payload, headers=headers)

if response.status_code == 200:
    access_token = response.json()['access_token']
    
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        show_data = response.json()
        print(show_data)
    else:
        print(f"Error: Unable to retrieve show information. Status code {response.status_code}")
else:
    print(f"Error: Unable to retrieve access token. Status code {response.status_code}")


{'href': 'https://api.spotify.com/v1/shows/4rOoJ6Egrf8K2IrywzwOMk/episodes?offset=1&limit=20', 'items': [{'audio_preview_url': 'https://podz-content.spotifycdn.com/audio/clips/5l5fBA3ukfHiHG5IxoTzEJ/clip_2487960_2532720.mp3', 'description': 'Dale Brisby is a cowboy, bullrider, rancher, YouTuber, and the star of Netflix\'s "How to Be a Cowboy." www.rodeotime.com https://www.youtube.com/@DaleBrisbyBullRider', 'duration_ms': 10859731, 'explicit': False, 'external_urls': {'spotify': 'https://open.spotify.com/episode/13LagQ3SH42tisSE2cT1gj'}, 'href': 'https://api.spotify.com/v1/episodes/13LagQ3SH42tisSE2cT1gj', 'html_description': '<p>Dale Brisby is a cowboy, bullrider, rancher, YouTuber, and the star of Netflix&#39;s &#34;How to Be a Cowboy.&#34;</p><br/><p>www.rodeotime.com</p><br/><p><a href="https://www.youtube.com/&#64;DaleBrisbyBullRider" rel="nofollow">https://www.youtube.com/&#64;DaleBrisbyBullRider</a></p>', 'id': '13LagQ3SH42tisSE2cT1gj', 'images': [{'height': 640, 'url': 'https:/

In [ ]:
show_data["items"][0]

{'audio_preview_url': 'https://podz-content.spotifycdn.com/audio/clips/5l5fBA3ukfHiHG5IxoTzEJ/clip_2487960_2532720.mp3',
 'description': 'Dale Brisby is a cowboy, bullrider, rancher, YouTuber, and the star of Netflix\'s "How to Be a Cowboy." www.rodeotime.com https://www.youtube.com/@DaleBrisbyBullRider',
 'duration_ms': 10859731,
 'explicit': False,
 'external_urls': {'spotify': 'https://open.spotify.com/episode/13LagQ3SH42tisSE2cT1gj'},
 'href': 'https://api.spotify.com/v1/episodes/13LagQ3SH42tisSE2cT1gj',
 'html_description': '<p>Dale Brisby is a cowboy, bullrider, rancher, YouTuber, and the star of Netflix&#39;s &#34;How to Be a Cowboy.&#34;</p><br/><p>www.rodeotime.com</p><br/><p><a href="https://www.youtube.com/&#64;DaleBrisbyBullRider" rel="nofollow">https://www.youtube.com/&#64;DaleBrisbyBullRider</a></p>',
 'id': '13LagQ3SH42tisSE2cT1gj',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/ab6765630000ba8a4e4f8b85965be1530a381213',
   'width': 640},
  {'height': 300

In [ ]:
for episode in show_data["items"]:
    description = episode['description']
    print(f"Episode Description: {description}\n")

print(len(show_data["items"]))

Episode Description: Dale Brisby is a cowboy, bullrider, rancher, YouTuber, and the star of Netflix's "How to Be a Cowboy." www.rodeotime.com https://www.youtube.com/@DaleBrisbyBullRider

Episode Description: David Blaine is an illusionist, endurance artist, and extreme performer. His new residency, "Impossible," is scheduled to begin at Encore Theater at Wynn Las Vegas on New Years Eve weekend.  www.davidblaine.com

Episode Description: Tim Kennedy is a Green Beret, retired UFC fighter, founder of Sheepdog Response, and co-founder of Save our Allies: an organization dedicated to rescuing and relocating Americans and allies in war-torn environments.  www.timkennedy.com 

Episode Description: Elon Musk is a business magnate, designer, and engineer. His portfolio of businesses include Tesla, Inc., SpaceX, Neuralink, X, and many others.https://twitter.com/elonmusk

Episode Description: Danny and Michael Philippou are the directors of the horror film "Talk to Me," available now on streamin

In [ ]:
## Fun and games with Anders

import requests
import base64

def extract_show_id_from_url(url):
    # Check if the URL is a valid Spotify show URL
    if 'open.spotify.com/show/' in url:
        # Split the URL by '/'
        parts = url.split('/')
        
        # The show ID is the last part
        show_id = parts[-1]
        return show_id
    else:
        return None


client_id='aa0207fb75aa45eaba50df2d8402a464'
client_secret='42e3594f4e86467a89e27bc2f1d32977'

#show_id = '4rOoJ6Egrf8K2IrywzwOMk'
url = f'https://api.spotify.com/v1/search?q=remaster%2520artist%3ATony%2520Hinchcliffe&type=show&market=US'
#url = 'https://api.spotify.com/v1/search?q=JRE&type=show'
auth_header = base64.b64encode(f'{client_id}:{client_secret}'.encode()).decode('utf-8')

headers = {
    'Authorization': f'Basic {auth_header}'
}

payload = {
    'grant_type': 'client_credentials'
}
response = requests.post('https://accounts.spotify.com/api/token', data=payload, headers=headers)

if response.status_code == 200:
    access_token = response.json()['access_token']
    
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        show_data = response.json()
        print(show_data)
    else:
        print(f"Error: Unable to retrieve show information. Status code {response.status_code}")
else:
    print(f"Error: Unable to retrieve access token. Status code {response.status_code}")


{'shows': {'href': 'https://api.spotify.com/v1/search?query=remaster%2520artist%3ATony%2520Hinchcliffe&type=show&market=US&offset=0&limit=20', 'items': [{'available_markets': ['US'], 'copyrights': [{'text': 'LibriVox 1884', 'type': 'C'}], 'description': 'Author(s): Samuel L. Clemens\nNarrator(s): LibriVox Community\n\n<p><em>Adventures of Huckleberry Finn</em> (1884) by Mark Twain is one of the truly great American novels, beloved by children, adults, and literary critics alike.  The book tells the story of “Huck” Finn (first introduced as Tom Sawyer’s sidekick in <em>The Adventures of Tom Sawyer</em>), his friend Jim, and their journey down the Mississippi River on a raft.  Both are on the run, Huck from his drunk and abusive father, and Jim as a runaway slave.</p><p>As Huck and Jim drift down the river, they meet many colorful characters and have many great adventures.  The true heart of the story, however, is the friendship between Huck and Jim.  A constant theme throughout the book

In [8]:
for shows in show_data["shows"]["items"]:
    print(shows["name"])

20 minut z Kamizelą
Monologato
Pride and Prejudice
Book of Basketball 2.0
AG University
20 Something The Podcast
Desert Island Discs: Archive 1996-2000
圆桌派
Bledsoe Said So
Hoaxed
Crying in H Mart: A Memoir
Frankenstein
英検1級面接・攻略ポイント20
Were You Raised By Wolves?
Juan María Gallardo 20H: Evangelios Narrados: 5A-
UK True Crime Podcast
Excuse Me, That's Illegal
The Supreme Court: Oral Arguments
دورة تأسيس وعي المسلم المعاصر 1 | م.أيمن عبد الرحيم | صيف 2017
Juan María Gallardo 20F: Evangelios Narrados: 4A-


In [ ]:
print(show_data["shows"]["items"][0]["id"])

77iUVnajPyEWPhY6LxjpzM


In [ ]:
## Fun and games with Anders

import requests
import base64

def extract_show_id_from_url(url):
    if 'open.spotify.com/show/' in url:
        parts = url.split('/')
        
        show_id = parts[-1]
        return show_id
    else:
        return None


client_id='aa0207fb75aa45eaba50df2d8402a464'
client_secret='42e3594f4e86467a89e27bc2f1d32977'

show_id = '77iUVnajPyEWPhY6LxjpzM'
url = f'https://api.spotify.com/v1/shows/{show_id}/episodes?offset=1&limit=20'

auth_header = base64.b64encode(f'{client_id}:{client_secret}'.encode()).decode('utf-8')

headers = {
    'Authorization': f'Basic {auth_header}'
}

payload = {
    'grant_type': 'client_credentials'
}
response = requests.post('https://accounts.spotify.com/api/token', data=payload, headers=headers)

if response.status_code == 200:
    access_token = response.json()['access_token']
    
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        show_data = response.json()
        print(show_data)
    else:
        print(f"Error: Unable to retrieve show information. Status code {response.status_code}")
else:
    print(f"Error: Unable to retrieve access token. Status code {response.status_code}")


{'href': 'https://api.spotify.com/v1/shows/77iUVnajPyEWPhY6LxjpzM/episodes?offset=1&limit=20', 'items': [{'audio_preview_url': 'https://podz-content.spotifycdn.com/audio/clips/7xdeZPWGSxqAZAof1UsYWM/clip_294338_354064.mp3', 'description': 'Yannis Pappas, Ryan Sickler, Mike Feeney, Kam Patterson, Paul Deemer, D Madness, Michael A. Gonzales, Hans Kim, William Montgomery, Jon Deas, Matthew Muehling, Joe White, Kristie Nova, Yoni, Tony Hinchcliffe, Brian Redban – 10/23/2023THIS EPISODE IS SPONSORED BY:Get 50% off of Factor & support the show at https://www.factormeals.com/KILLTONY50 & use code KILLTONY50—Support the show & get $80 off your first month of therapy at https://www.talkspace.com/tony—SHEATH UNDERWEAR\xa0– Go to\xa0SHEATHUNDERWEAR.COM\xa0and use promo code “TONY” for 20% off!—KACHAVA – Nutrient-Rich Superfoods – 10% OFF FIRST ORDER – https://www.kachava.com/collections/kill-tony-podcast—DISPLATE.COM – USE CODE “TONY” FOR UP TO 30% OFF!', 'duration_ms': 7734048, 'explicit': False

In [ ]:
for episode in show_data["items"]:
    description = episode['description']
    print(f"Episode Description: {description}\n")

print(len(show_data["items"]))

Episode Description: Yannis Pappas, Ryan Sickler, Mike Feeney, Kam Patterson, Paul Deemer, D Madness, Michael A. Gonzales, Hans Kim, William Montgomery, Jon Deas, Matthew Muehling, Joe White, Kristie Nova, Yoni, Tony Hinchcliffe, Brian Redban – 10/23/2023THIS EPISODE IS SPONSORED BY:Get 50% off of Factor & support the show at https://www.factormeals.com/KILLTONY50 & use code KILLTONY50—Support the show & get $80 off your first month of therapy at https://www.talkspace.com/tony—SHEATH UNDERWEAR – Go to SHEATHUNDERWEAR.COM and use promo code “TONY” for 20% off!—KACHAVA – Nutrient-Rich Superfoods – 10% OFF FIRST ORDER – https://www.kachava.com/collections/kill-tony-podcast—DISPLATE.COM – USE CODE “TONY” FOR UP TO 30% OFF!

Episode Description: Kam Patterson, David Jolly, Paul Deemer, D Madness, Michael A. Gonzales, Hans Kim, William Montgomery, Jon Deas, Matthew Muehling, Joe White, Kristie Nova, Yoni, Tony Hinchcliffe, Brian Redban – 10/02/2023THIS EPISODE IS SPONSORED BY:Don’t miss out 

In [9]:
## Fun and games with Anders

import requests
import base64

def extract_show_id_from_url(url):
    # Check if the URL is a valid Spotify show URL
    if 'open.spotify.com/show/' in url:
        # Split the URL by '/'
        parts = url.split('/')
        
        # The show ID is the last part
        show_id = parts[-1]
        return show_id
    else:
        return None


client_id='aa0207fb75aa45eaba50df2d8402a464'
client_secret='42e3594f4e86467a89e27bc2f1d32977'

#show_id = '4rOoJ6Egrf8K2IrywzwOMk'
url = f'https://api.spotify.com/v1/search?q=remaster%2520track%3AKill%2520Tony&type=show&market=US'
#url = 'https://api.spotify.com/v1/search?q=JRE&type=show'
auth_header = base64.b64encode(f'{client_id}:{client_secret}'.encode()).decode('utf-8')

headers = {
    'Authorization': f'Basic {auth_header}'
}

payload = {
    'grant_type': 'client_credentials'
}
response = requests.post('https://accounts.spotify.com/api/token', data=payload, headers=headers)

if response.status_code == 200:
    access_token = response.json()['access_token']
    
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        show_data = response.json()
        print(show_data)
    else:
        print(f"Error: Unable to retrieve show information. Status code {response.status_code}")
else:
    print(f"Error: Unable to retrieve access token. Status code {response.status_code}")


{'shows': {'href': 'https://api.spotify.com/v1/search?query=remaster%2520track%3AKill%2520Tony&type=show&market=US&offset=0&limit=20', 'items': [{'available_markets': ['AD', 'AE', 'AG', 'AL', 'AM', 'AR', 'AT', 'AU', 'BA', 'BB', 'BE', 'BF', 'BG', 'BH', 'BJ', 'BO', 'BR', 'BS', 'BT', 'BW', 'BZ', 'CA', 'CH', 'CL', 'CO', 'CR', 'CV', 'CW', 'CY', 'CZ', 'DE', 'DK', 'DM', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'FI', 'FJ', 'FM', 'FR', 'GB', 'GD', 'GE', 'GH', 'GM', 'GR', 'GT', 'GW', 'GY', 'HK', 'HN', 'HR', 'HT', 'HU', 'ID', 'IE', 'IL', 'IN', 'IS', 'IT', 'JM', 'JO', 'JP', 'KE', 'KI', 'KN', 'KR', 'KW', 'LB', 'LC', 'LI', 'LR', 'LS', 'LT', 'LU', 'LV', 'MA', 'MC', 'ME', 'MG', 'MH', 'MK', 'ML', 'MT', 'MU', 'MV', 'MW', 'MX', 'MY', 'MZ', 'NA', 'NE', 'NG', 'NI', 'NL', 'NO', 'NR', 'NZ', 'OM', 'PA', 'PE', 'PG', 'PH', 'PL', 'PS', 'PT', 'PW', 'PY', 'QA', 'RO', 'RS', 'SA', 'SB', 'SC', 'SE', 'SG', 'SI', 'SK', 'SL', 'SM', 'SN', 'SR', 'ST', 'SV', 'TH', 'TL', 'TN', 'TO', 'TR', 'TT', 'TV', 'TW', 'UA', 'US', 'UY', 'VC'

In [11]:
show_data["shows"]["items"][0]

{'available_markets': ['AD',
  'AE',
  'AG',
  'AL',
  'AM',
  'AR',
  'AT',
  'AU',
  'BA',
  'BB',
  'BE',
  'BF',
  'BG',
  'BH',
  'BJ',
  'BO',
  'BR',
  'BS',
  'BT',
  'BW',
  'BZ',
  'CA',
  'CH',
  'CL',
  'CO',
  'CR',
  'CV',
  'CW',
  'CY',
  'CZ',
  'DE',
  'DK',
  'DM',
  'DO',
  'DZ',
  'EC',
  'EE',
  'EG',
  'ES',
  'FI',
  'FJ',
  'FM',
  'FR',
  'GB',
  'GD',
  'GE',
  'GH',
  'GM',
  'GR',
  'GT',
  'GW',
  'GY',
  'HK',
  'HN',
  'HR',
  'HT',
  'HU',
  'ID',
  'IE',
  'IL',
  'IN',
  'IS',
  'IT',
  'JM',
  'JO',
  'JP',
  'KE',
  'KI',
  'KN',
  'KR',
  'KW',
  'LB',
  'LC',
  'LI',
  'LR',
  'LS',
  'LT',
  'LU',
  'LV',
  'MA',
  'MC',
  'ME',
  'MG',
  'MH',
  'MK',
  'ML',
  'MT',
  'MU',
  'MV',
  'MW',
  'MX',
  'MY',
  'MZ',
  'NA',
  'NE',
  'NG',
  'NI',
  'NL',
  'NO',
  'NR',
  'NZ',
  'OM',
  'PA',
  'PE',
  'PG',
  'PH',
  'PL',
  'PS',
  'PT',
  'PW',
  'PY',
  'QA',
  'RO',
  'RS',
  'SA',
  'SB',
  'SC',
  'SE',
  'SG',
  'SI',
  'SK',
  'SL',
  '

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=80392662-d35f-44b5-a185-8bbb8141c971' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>